# **Beijing Air Quality Data Analysis and Application**

#  **1.Data Preprocessing and Exploratory Data Analysis (EDA)**

**Objective:** Clean the data, understand the data distribution and characteristics, and identify potential issues and data trends.

**Methods Used:**

	•	Data Cleaning: Handle missing values, duplicates, and outliers.
	•	pandas: dropna(), fillna(), drop_duplicates()
	•	numpy: Data processing and manipulation
	•	Data Visualization: Plot time series, box plots, histograms for various pollutants and meteorological parameters.
	•	matplotlib, seaborn

# Loading data

In [ ]:
import pandas as pd


# read all tables
df = pd.DataFrame()

for i in range(0, 12):
    df = pd.concat([df, pd.read_csv(f"/kaggle/input/beijingclimatedata/PRSA_Data_{i}.csv")], axis = 0)

data = df.copy()

print("There are {} missing values in this dataset".format(data.isnull().sum().sum()))
print('Number of instances = %d' % (data.shape[0]))
print('Number of attributes = %d' % (data.shape[1]))

# Count the number of rows with missing values
rows_with_missing = data.isnull().any(axis=1).sum()
print(f"Rows with missing values: {rows_with_missing}")

# Count the number of columns with missing values
columns_with_missing = data.isnull().any(axis=0).sum()
print(f"Columns with missing values: {columns_with_missing}")

print('\nNumber of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col,data[col].isna().sum()))

data.columns

# remove the empty data rows

In [ ]:
# remove the empty data rows
data = data.dropna()

# Count the number of rows with missing values
rows_with_missing = data.isnull().any(axis=1).sum()
print(f"Rows with missing values: {rows_with_missing}")

# Count the number of columns with missing values
columns_with_missing = data.isnull().any(axis=0).sum()
print(f"Columns with missing values: {columns_with_missing}")

print('Number of instances = %d' % (data.shape[0]))

# data.describe()
# data.head()
data.dtypes.value_counts()

# Convert Time and Wind Direction Column

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert Time Columns
data['datetime'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
data.set_index('datetime', inplace=True)

# Convert Wind Direction Column
direction_map = {
    'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5,
    'E': 90, 'ESE': 112.5, 'SE': 135, 'SSE': 157.5,
    'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
    'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5
}

data['wd_numeric'] = data['wd'].map(direction_map)

# Remove the string column, there are restrictions when taking the mean later.
data = data.drop(columns = ['wd', 'station'], axis=1)

# Plot Time Series

In [ ]:
# downsample the data，take one data point per day
df_daily = data.resample('D').mean()

plt.figure(figsize=(14, 8))
plt.plot(df_daily['PM2.5'], label='PM2.5')
plt.plot(df_daily['PM10'], label='PM10')
plt.plot(df_daily['SO2'], label='SO2')
plt.plot(df_daily['NO2'], label='NO2')
plt.xlabel('Date')
plt.ylabel('Concentration (µg/m³)')
plt.title('Daily Mean Time Series of Air Pollutants')
plt.legend()
plt.show()

# Plot meteorological parameters

In [ ]:
plt.figure(figsize=(14, 8))
plt.plot(df_daily['TEMP'], label='Temperature (°C)')
plt.plot(df_daily['RAIN'], label='Rainfall (mm)')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Daily Mean Time Series of Meteorological Parameters')
plt.legend()
plt.show()

# Plot Histograms

In [ ]:
# sample 10% of the data before plotting histograms
df_sample = data.sample(frac=0.1, random_state=1)

plt.figure(figsize=(14, 8))
df_sample[['PM2.5', 'PM10', 'SO2', 'NO2']].hist(bins=30, layout=(2, 2), figsize=(14, 8))
plt.suptitle('Histogram of Air Pollutants')
plt.show()

# Histograms for meteorological parameters

In [ ]:
plt.figure(figsize=(14, 8))
df_sample[['TEMP', 'RAIN']].hist(bins=30, layout=(1, 2), figsize=(14, 8))
plt.suptitle('Histogram of Meteorological Parameters')
plt.show()

# Plot Joint Distribution of Wind Speed and Wind Direction

In [ ]:
# a scatter plot
plt.figure(figsize=(14, 8))
sns.scatterplot(x='wd_numeric', y='WSPM', data=df_sample)
plt.title('Wind Speed vs Wind Direction')
plt.xlabel('Wind Direction (Degrees)')
plt.ylabel('Wind Speed (m/s)')
plt.show()

# **2. Time Series Analysis**
**Objective:** Analyze the trends and seasonal patterns of air pollutants over time.

**Methods Used:**

* Time Series Decomposition: Decompose into trend, seasonal, and residual components.
* statsmodels: seasonal_decompose
* ARIMA Model: Forecast future air pollutant concentrations.
* statsmodels: ARIMA

# Time Series Decomposition: Decompose into trend, seasonal, and residual components.

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_model import ARMA

# data.columns

data['datetime'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
data.set_index('datetime', inplace=True)

# Downsample to daily data
df_daily = data.resample('D').mean()

# time series decomposition
decomposition = seasonal_decompose(df_daily['PM2.5'], model='additive')
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plot decomposition results
plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(df_daily['PM2.5'], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal, label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

# ARIMA Model: Forecast future air pollutant concentrations.

In [ ]:
# Split training set and test set
train = df_daily['PM2.5'][:int(0.8*len(df_daily))]
test = df_daily['PM2.5'][int(0.8*len(df_daily)):]

# Fit ARIMA model
arima_model = ARIMA(train, order=(5,1,0))
arima_result = arima_model.fit()

# predict
arima_forecast = arima_result.forecast(steps=len(test))

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(train, label='Train')
plt.plot(test, label='Test')
plt.plot(arima_forecast, label='ARIMA Forecast')
plt.legend(loc='best')
plt.show()

# **3. Clustering Analysis**
**Objective:** Classify air quality in different districts or time periods, identifying similar regions or periods.

**Methods Used:**

* K-means Clustering: Cluster air quality data from different districts. 
* scikit-learn: KMeans 
* Hierarchical Clustering: Cluster air quality data from different time periods. 
* scikit-learn: AgglomerativeClustering

# use the elbow method to figure out how many clusters we have

In [ ]:
from sklearn import cluster

# Select features for clustering
features = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
X = data[features].dropna()

numClusters = [1,2,3,4,5,6,7,8,9,10]
SSE = []
for k in numClusters:
    k_means = cluster.KMeans(n_clusters=k)
    k_means.fit(X)
    SSE.append(k_means.inertia_)
plt.plot(numClusters, SSE)
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')

# use the silhouette analysis technique to determe the optimal value of k

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Select features for clustering
features = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
X = df_daily[features]

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)
    
    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])
    
    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =", 
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )
    
    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)
    
    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        
        ith_cluster_silhouette_values.sort()
        
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )
        
        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10 # 10 for the 0 samples
        
    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")
    
    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    
    ax1.set_yticks([]) # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X.iloc[:, 0], X.iloc[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k")
    
    # Labelling the clusters
    centers = clusterer.cluster_centers_
    
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )
    
    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")
        
    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Latitude")
    ax2.set_ylabel("Longitude")
    
    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters= %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering

# Select features for clustering
features = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
X = data[features].dropna()

# Perform K-means clustering
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X)
data['kmeans_cluster'] = kmeans.labels_

# Visualize clustering results
plt.figure(figsize=(14, 8))
sns.scatterplot(x='PM2.5', y='PM10', hue='kmeans_cluster', data=data, palette='viridis')
plt.title('K-means Clustering of Air Quality Data')
plt.show()

In [ ]:
# Perform cluster analysis on different time periods
# Take the daily average
df_daily = data.resample('D').mean().dropna()

# Select features for clustering
X_daily = df_daily[features]

# Perform hierarchical clustering
hc = AgglomerativeClustering(n_clusters=2)
hc_labels = hc.fit_predict(X_daily)
df_daily['hc_cluster'] = hc_labels

# Visualize clustering results
plt.figure(figsize=(14, 8))
sns.scatterplot(x='PM2.5', y='PM10', hue='hc_cluster', data=df_daily, palette='viridis')
plt.title('Hierarchical Clustering of Daily Mean Air Quality Data')
plt.show()

In [ ]:
from sklearn.decomposition import PCA

# Standardize features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform PCA dimensionality reduction
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Perform K-means clustering
kmeans_pca = KMeans(n_clusters=2, random_state=0)
kmeans_pca.fit(X_pca)
data['kmeans_pca_cluster'] = kmeans_pca.labels_

# Visualize clustering results
plt.figure(figsize=(14, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=data['kmeans_pca_cluster'], palette='viridis')
plt.title('K-means Clustering with PCA')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

# **4. Association Rule Mining**
**Objective:** Discover association patterns between air pollutants and between air pollutants and meteorological parameters.

**Methods Used:** 

* Apriori Algorithm: Mine frequent itemsets and association rules.
* mlxtend: apriori, association_rules

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

# 转换时间列
data['datetime'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
data.set_index('datetime', inplace=True)


# 数据离散化

# 选择需要离散化的列
columns_to_discretize = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN']

# 使用pd.qcut进行离散化
for col in columns_to_discretize:
    data[col + '_bin'] = pd.qcut(data[col], q=2, labels=False, duplicates='drop')

# 选择离散化后的列
discretized_columns = [col + '_bin' for col in columns_to_discretize]

# 转换为one-hot编码
df_onehot = pd.get_dummies(data[discretized_columns])

# 检查转换结果
print(df_onehot.head())

# 挖掘频繁项集
frequent_itemsets = apriori(df_onehot, min_support=0.05, use_colnames=True)

# 挖掘关联规则
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 显示关联规则
# print(rules)

# 选择提升度最大的前10条规则进行可视化
top_rules = rules.sort_values('lift', ascending=False).head(10)

plt.figure(figsize=(10, 6))
plt.barh(range(len(top_rules)), top_rules['lift'], align='center')
plt.yticks(range(len(top_rules)), top_rules['antecedents'].astype(str) + ' -> ' + top_rules['consequents'].astype(str))
plt.xlabel('Lift')
plt.title('Top 10 Association Rules by Lift')
plt.show()

# **5. Classification Analysis**
**Objective:** Predict air quality levels (e.g., good, moderate, lightly polluted) for a specific district based on historical data.

**Methods Used:**

* Decision Tree: Build a classification model for air quality levels based on various pollutants and meteorological parameters.
* scikit-learn: DecisionTreeClassifier
* Support Vector Machine (SVM): Classify air quality levels.
* scikit-learn: SVC
* Random Forest: Enhance the accuracy and stability of the classification model.
* scikit-learn: RandomForestClassifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# 转换时间列
data['datetime'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
data.set_index('datetime', inplace=True)

#定义空气质量等级
def categorize_pm25(pm25):
    if pm25 <= 50:
        return 'Excellent'
    elif pm25 <= 100:
        return 'Good'
    elif pm25 <= 150:
        return 'Light Pollution'
    elif pm25 <= 200:
        return 'Moderate Pollution'
    elif pm25 <= 300:
        return 'Heavy Pollution'
    else:
        return 'Severe Pollution'

data['AQI_Category'] = data['PM2.5'].apply(categorize_pm25)

# 特征选择和数据分割
features = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
X = data[features]
y = data['AQI_Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 对特征进行标准化处理
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
# 构建和评估分类模型
# 决策树
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_scaled, y_train)
y_pred_dt = dt_classifier.predict(X_test_scaled)

print("Decision Tree Classification Report")
print(classification_report(y_test, y_pred_dt))


In [ ]:

# 支持向量机（SVM）
# svm_classifier = SVC(random_state=42)
# svm_classifier.fit(X_train_scaled, y_train)
# y_pred_svm = svm_classifier.predict(X_test_scaled)

# print("SVM Classification Report")
# print(classification_report(y_test, y_pred_svm))


# from sklearn.decomposition import PCA

# # Apply PCA
# pca = PCA(n_components=0.95)  # Retain 95% of variance
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_test_pca = pca.transform(X_test_scaled)

# svm_classifier = SVC(random_state=42)
# svm_classifier.fit(X_train_pca, y_train)
# y_pred_svm = svm_classifier.predict(X_test_pca)

# print("SVM (PCA) Classification Report")
# print(classification_report(y_test, y_pred_svm))

from sklearn.svm import SVC

# Using linear kernel
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train_scaled, y_train)
y_pred_svm = svm_classifier.predict(X_test_scaled)

print("SVM (Linear Kernel) Classification Report")
print(classification_report(y_test, y_pred_svm))


In [ ]:

# 随机森林
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_scaled, y_train)
y_pred_rf = rf_classifier.predict(X_test_scaled)

print("Random Forest Classification Report")
print(classification_report(y_test, y_pred_rf))


In [ ]:

# 可视化混淆矩阵
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=['Excellent', 'Good', 'Light Pollution', 'Moderate Pollution', 'Heavy Pollution', 'Severe Pollution'])
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Excellent', 'Good', 'Light Pollution', 'Moderate Pollution', 'Heavy Pollution', 'Severe Pollution'], yticklabels=['Excellent', 'Good', 'Light Pollution', 'Moderate Pollution', 'Heavy Pollution', 'Severe Pollution'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(title)
    plt.show()

plot_confusion_matrix(y_test, y_pred_dt, "Decision Tree Confusion Matrix")
# plot_confusion_matrix(y_test, y_pred_svm, "SVM Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix")